In [ ]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, default_data_collator
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig, PeftModel
import os
import glob
import re
from pathlib import Path


In [ ]:
#грузим файл
file_path = 'train.jsonl'
# качаем с huggingface модель
model_id = "google/gemma-3-1b-it"
 
# конфигурация для 4-битного квантования (QLoRA)
bnb_config = BitsAndBytesConfig(
	load_in_4bit=True,
	bnb_4bit_quant_type="nf4",
	bnb_4bit_compute_dtype=torch.float16
)
 
# модель с квантованием
model = AutoModelForCausalLM.from_pretrained(
	model_id,
	quantization_config=bnb_config,
	# если есть GPU nvidia на него модель
    device_map={"": "cuda:0"} if torch.cuda.is_available() else {"": "cpu"}
)
 
# токенизатор
tokenizer = AutoTokenizer.from_pretrained(model_id)
# токен для паддинга - это важный шаг
tokenizer.pad_token = tokenizer.eos_token
# Дополнительно, установите токен паддинга как токен конца предложения,
# чтобы избежать предупреждений.
tokenizer.padding_side = "right" # Хорошая практика для CausalLM
 
 
# создаем датасет из jsonl файла
ds = load_dataset("json", data_files="train.jsonl", split="train")


In [ ]:
# параметры обучения
training_args = TrainingArguments(
	output_dir="./gemma-finetuned",
	per_device_train_batch_size=1,
	gradient_accumulation_steps=4,
	learning_rate=2e-4,
	num_train_epochs=3,
	logging_steps=10,
	save_strategy="epoch",
	fp16=True if torch.cuda.is_available() else False, # fp16 работает только на CUDA
)
 
# LoRA конфиг
lora_config = LoraConfig(
	r=8,
	target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
	task_type="CAUSAL_LM",
)
 
trainer = SFTTrainer(
	model=model,
	args=training_args,
	# передаем сырой датасет.
	train_dataset=ds,
	peft_config=lora_config,
	# передаем токенизатор
	processing_class=tokenizer,
)
 
# запускаем файн-тюнинг
trainer.train()
 
print("Обучение успешно завершено!")


In [ ]:
base_model_id = "google/gemma-3-1b-it"
output_dir = "./gemma-finetuned"
merged_model_dir = "./gemma-finetuned-merged"
all_checkpoints = glob.glob(os.path.join(output_dir, 'checkpoint-*'))
if not all_checkpoints:
	raise FileNotFoundError(f"Не найдены папки 'checkpoint-' в каталоге {output_dir}. Убедитесь, что обучение завершилось.")
def extract_step(checkpoint_path):
	match = re.search(r'checkpoint-(\d+)', checkpoint_path)
	return int(match.group(1)) if match else -1
latest_checkpoint = max(all_checkpoints, key=extract_step)
lora_adapter_dir = latest_checkpoint
print(f"последний чекпоинт: {lora_adapter_dir}")


In [ ]:
print("Загрузка базовой модели...")
base_model = AutoModelForCausalLM.from_pretrained(
	base_model_id,
	torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
	device_map={"": "cuda:0"} if torch.cuda.is_available() else {"": "cpu"}
)
# загрузка токенизатора
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
# слияние адаптера LoRA
print("Слияние LoRA-адаптера с базовой моделью...")
model = PeftModel.from_pretrained(base_model, lora_adapter_dir)
merged_model = model.merge_and_unload()
# сохраняем объединенную модель в папку
os.makedirs(merged_model_dir, exist_ok=True)
merged_model.save_pretrained(merged_model_dir, safe_serialization=True)
tokenizer.save_pretrained(merged_model_dir)
print('Готово')
print(f"Файлы модели находятся в: {merged_model_dir}")


In [ ]:
# клонируем репозиторий llama.cpp для работы с моделями (здесь для компиляции)
!git clone https://github.com/ggerganov/llama.cpp.git
# установим зависимости - по идее не стоит, ставит слишком много всякого,
# установок в начале работы должно хватить
# REQURIMENTS_LLAMA = str(Path.cwd()) + "\\llama.cpp\\requirements.txt"
# !pip install -r "{REQURIMENTS_LLAMA}"


In [ ]:
# указываем пути до модели, выходного  скрипта конвертации
merged_model_dir = Path.cwd() / Path("./gemma-finetuned-merged")
HF_MODEL_PATH_STR = str(merged_model_dir)
GGUF_MODEL_PATH_STR = str(merged_model_dir / "gemma-finetuned-temp.gguf")
CONVERT_SCRIPT_PATH = "./llama.cpp/convert_hf_to_gguf.py"
 
# запуск скрипта конвертации
# --outtype f16 - это базовый формат при конвертации для ollama
!python {CONVERT_SCRIPT_PATH} "{HF_MODEL_PATH_STR}" --outtype f16 --outfile "{GGUF_MODEL_PATH_STR}"
